import pandas as pd 
from collections import defaultdict
import math
import copy
import json
import ast
data= pd.read_csv('input_p.csv')

In [1]:
import pandas as pd
import numpy as np
import random
import copy
import math
from collections import defaultdict

randomlist = []
df_demand = {}
random.seed(2) # Use the same random seed
for i in range(0,50):   # 20 depots
    n = random.randint(100,150) # Depot Capacity
    randomlist.append(n)

for i in range(50):
    df_demand['D'+str(i+1)] = randomlist[i]

sum = 0
for (k,v) in df_demand.items():
    sum = sum + v
    
df_supply = {}
for i in range(sum):
    df_supply[i] = 1.0
    
np.random.seed(2) # Use the same random seed
df_dkm = pd.DataFrame(np.random.uniform(0.5,20, size=(sum,50))).astype(float)

random.seed(2) # Use the same random seed
age_b = []
for i in range(sum):
    n = random.randint(1,3) #Only to generate integer values inclusive of 1 & 3
    age_b.append(n)
    
age = {}
for i in range(len(age_b)):
    age[i] = age_b[i] 
    
kpl = []
co2 = []
random.seed(2) # Use the same random seed
for (k,v) in age.items():
    if (v==1):
        n = random.uniform(5.1,6)
        m = round(random.uniform(515, 524), 2)
        kpl.append(n)
        co2.append(m)
    if (v==2):
        n = random.uniform(4.1,5)
        m = round(random.uniform(525, 534), 2)
        kpl.append(n)
        co2.append(m)
    if (v==3):
        n = random.uniform(3.1,4)
        m = round(random.uniform(535, 540), 2)
        kpl.append(n)
        co2.append(m)
        
fc_kpl = {}
for i in range(sum):
    fc_kpl[i] =(111/ kpl[i])

co2_c = {}
for i in range(sum):
    co2_c[i] =(0.00118642*co2[i])
    
random.seed(2) # Use the same random seed
doc = {}
for i in range(0,50):
    n = random.uniform(50,100)
    doc[i] = n
    
df_tdkom = pd.DataFrame(np.zeros(df_dkm.shape))
for i in range(len(df_dkm.columns)):
    for j in range(len(df_dkm.index)):
        df_tdkom[i][j] = df_dkm[i][j]*fc_kpl[j] + df_dkm[i][j]*co2_c[j] + doc[i]
#df_tdkom

df_tdkom.columns = [("D"+str(i)) for i in range(1,51)]

In [2]:
dk = df_tdkom.transpose()

costs_t = dk.to_dict()
#costs_t

colsn = sorted(df_demand.keys())

# print(supply['S1'])
# break

costs1=copy.deepcopy(costs_t)
costs2=copy.deepcopy(costs_t)
costs3=copy.deepcopy(costs_t)
for i in df_supply:
    mi=min(costs_t[i].values())
    # print(costs[i])
    # print(mi)
    for j in costs2[i]:
        costs2[i][j]-=mi
# print(costs2)
for i in df_demand :
    mi=10000
    for j in df_supply:
        if costs_t[j][i]<mi :
            mi=costs_t[j][i]
    for j in df_supply:
        costs3[j][i]=costs3[j][i]-mi 
# print(costs3)

for i in df_demand:
    for j in df_supply:
        costs_t[j][i]= costs2[j][i]+costs3[j][i]
# print(costs_t)


res = dict((k, defaultdict(int)) for k in costs_t)
g = {}
for x in df_supply:
    # print(x)
    # print(costs_t[x])
    g[x] = sorted(costs_t[x].keys(), key=lambda g: costs_t[x][g])
for x in df_demand:
    g[x] = sorted(costs_t.keys(), key=lambda g: costs_t[g][x])

In [3]:
while g:
    d = {}
    # print(df_supply,dic)
    for x in df_demand:
        d[x] = (costs_t[g[x][1]][x] - costs_t[g[x][0]][x])/df_demand[x] if len(g[x]) > 1 else (costs_t[g[x][0]][x])/df_demand[x]
    s = {}
    for x in df_supply:
        s[x] = (costs_t[x][g[x][1]] - costs_t[x][g[x][0]])/df_supply[x]  if len(g[x]) > 1 else costs_t[x][g[x][0]]/df_supply[x]

    f = max(d, key=lambda n: d[n])
    t = max(s, key=lambda n: s[n])
    t, f = (f, g[f][0]) if d[f] >= s[t] else (g[t][0], t)
    v = min(df_supply[f], df_demand[t])
    # print(f,t)
    # print(v)
    res[f][t] += v
    df_demand[t] -= v

    if df_demand[t] == 0:
        for k, n in df_supply.items():
            if n != 0:
                g[k].remove(t)
        del g[t]
        del df_demand[t]
    df_supply[f] -= v
    if df_supply[f] == 0:
        for k, n in df_demand.items():
            if n != 0:
                g[k].remove(f)
        del g[f]
        del df_supply[f]

# print("G",g)
cost = 0
# cols = sorted(df_demand.keys())
# print(costs)
for g in sorted(costs1):
    # print (g, " ",)
    # print("S")
    for n in colsn:
        y = res[g][n]
        # print("YESS",y)
        if y != 0:
            pass
            # print (y,)
        cost += y * costs1[g][n]
        # print ("  ",)
    # print(" ")

In [4]:
df_tdkom

,D1,D2,D3,D4,D5,D6,D7,D8,D9,D10,...,D41,D42,D43,D44,D45,D46,D47,D48,D49,D50
0,271.033760,116.742341,268.714152,227.223283,259.142814,220.380876,169.904294,307.414469,202.366649,190.090822,...,459.720650,286.740918,411.109822,236.863664,320.541739,228.120757,261.214579,386.788673,283.681443,444.038065
1,344.062232,143.949501,222.223339,433.019712,278.427619,109.634789,201.385026,105.504063,520.871470,510.857476,...,234.599831,118.631054,397.869833,394.273808,201.930735,249.240151,377.017492,382.697702,157.822864,468.604876
2,404.751140,158.347904,394.498988,350.028679,406.583832,304.119699,177.627808,426.000547,316.234135,222.377219,...,343.647017,160.952105,409.830280,191.877950,391.197791,317.270308,402.623620,107.485831,170.548334,355.439868
3,151.287208,123.958713,233.678473,345.013713,128.563383,129.937405,116.154276,264.219138,408.224904,186.017008,...,544.595419,149.859363,392.565228,244.995305,205.102086,253.650907,537.277403,367.310779,518.306942,420.432607
4,306.149534,307.665206,149.500934,66.227650,267.149740,319.727597,440.266133,343.500203,423.540901,456.733981,...,248.870012,170.140533,386.939425,295.925378,210.847416,342.011026,485.576651,193.279319,395.715136,388.126040
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6142,253.228766,388.565647,388.923975,74.751434,377.231616,352.517658,393.199773,136.413137,410.586337,419.466765,...,445.477496,473.802240,130.564595,300.726271,264.721610,308.590983,157.182073,125.193578,181.978377,297.885254
6143,123.895328,380.719987,536.052706,500.822422,338.653403,492.481639,562.387318,105.328856,277.912423,384.238402,...,348.627124,534.507096,208.367475,451.198122,166.289639,88.256243,135.747042,294.030531,380.437911,572.476479
6144,134.685505,470.747882,283.238104,499.686076,459.537421,280.878710,256.942949,124.153362,251.464991,373.570832,...,523.334180,105.491784,421.030664,531.304999,415.715486,502.712076,473.534698,195.143689,351.547851,485.008526
6145,532.165167,156.018234,680.001055,243.162668,345.269666,370.405743,643.256159,242.365013,261.163664,119.183728,...,174.150439,289.806455,507.783586,625.594788,119.823104,290.265288,186.906054,517.990949,632.227702,687.390534


In [5]:
cost

620930.1986466715